# Customer Support Analysis using CrewAI Agents

In [114]:
import warnings
warnings.filterwarnings('ignore')
import os
import yaml
from crewai import Agent, Task, Crew, LLM
from dotenv import load_dotenv
import pandas as pd

### Loading Configurations

In [115]:
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)


In [116]:
agents_config = configs['agents']
tasks_config = configs['tasks']

### Loading Support Ticket Feedback

In [117]:
filename = './support.csv'
data = pd.read_csv(filename)
data.head()

,ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
0,T0001,C0511,API Issue,I'm pleased with how my issue was handled. Tha...,High,2023-03-25,240,927,4,I'm pleased with how my issue was handled. Tha...,A004,True
1,T0002,C0729,Login Issue,Excellent service! The agent went above and be...,Low,2023-04-06,223,534,5,The problem still persists. Not resolved yet.,A004,False
2,T0003,C0439,Report Generation,Resolution was satisfactory but could be impro...,Low,2023-04-25,214,592,1,The issue was escalated quickly which was appr...,A004,False
3,T0004,C0100,Data Import,Agent was very helpful and polite. Great service!,High,2023-02-20,110,864,2,The issue was resolved quickly. Very satisfied!,A003,True
4,T0005,C0258,Feature Request,My issue was handled well but follow-up could ...,Medium,2023-01-18,151,193,2,The problem still persists. Not resolved yet.,A004,False


In [118]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support.csv')

### Check if docker is running to enable code generation by agents

In [127]:
import subprocess
try:
    result = subprocess.run(["docker", "info"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Docker is running.")
except subprocess.CalledProcessError as e:
    print("Docker is not running or accessible.")
except FileNotFoundError:
    print("Docker is not installed.")

Docker is running.


### Assemble Agents, Tasks and Crew

In [120]:
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool],
  
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool],
  
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=True,
  
)

# Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)

support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


2024-11-23 13:03:24,728 - 8429515328 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


### Test crew performance

In [122]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2024-11-23 13:03:28,163 - 8429515328 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Thought: I need to analyze the support tickets to generate actionable suggestions. To do this, I will read the content of the support.csv file to gather information on issue types, historical data, and customer feedback.
## Using tool: Read a file's content
## Tool Input: 
"{}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  8.5  │    8.5     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.2  │    8.2     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  9.3  │    9.3     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │ 10.0  │    10.0    │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.90  │    8.9     │                     │  │  ┃
┃ Execution Time (s) │  185  │    185     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

### Iteratively provide feedback on each task response

In [125]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

2024-11-23 13:30:11,229 - 8429515328 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
T0001,C0511,API Issue,"I'm pleased with how my issue was handled. Thanks!",High,2023-03-25,240,927,4,"I'm pleased with how my issue was 

### Test crew performance after feedback incorporation

In [126]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2024-11-23 13:38:58,406 - 8429515328 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Thought: I need to gather data from the support tickets to analyze the issues reported, resolution times, customer feedback, and satisfaction ratings. This will help me generate actionable suggestions for resolving each ticket efficiently.
## Using tool: Read a file's content
## Tool Input: 
"{}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submi

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  8.5  │    8.5     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.2  │    8.2     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  9.3  │    9.3     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │ 10.0  │    10.0    │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.95  │    8.9     │                     │  │  ┃
┃ Execution Time (s) │  435  │    435     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛